In [ ]:
Install libraries

%pip install tensorflow
# deep learning library used to creat sequential models

!pip install pandas
# read data in tabular format since it is in excel or csv format

!pip install matplotlib
#help with plotting
!pip install scikit-learn
#used basically for matrix
Import requierd libraries
import os #help to work with different file paths or navigate through different file paths
import pandas as pd
import tensorflow as tf
import numpy as np
Bring in our data
df = pd.read_csv(  # used to read in our csv
    os.path.join(r'C:\Users\FALGUNI\OneDrive\Documents\Hate comment detection\jigsaw-toxic-comment-classification-challenge\train.csv.zip'))
    #pass the path of the training data we are going to use

df.head()
#to read data from start of our file 
df.tail()
#to read data from the end of our file
df.iloc[6]['comment_text']
#to read particular comment
df[df.columns[2:]].iloc[6]
# to look at perticular comment attribute values
Preprocessing of the data

Tokenizing the data --> translating the sentences into our own secret language that only deep learning network understand
i.e. convert each word in the sentence into a unique identifier so that each word maps to a number
from tensorflow.keras.layers  import TextVectorization
# using tectvectorization to tokenize our text
split the data into comments and features
x = df['comment_text']
y = df[df.columns[2:]].values
#  we can customize our output as we need to display only the index values use df.columns this will list all columns of table. If we need all values for particular column then type df['comment_text'] this will give us all the comments

#to get all columns from 2nd and use .values to represent in the form of an array. .values will give us the format which we can pass to out tenserflow deep learning model.

#so under X we have all our bad comments and under Y we have all our labels

df[df.columns[2:]].values
#printing all columns that conatain label.
MAX_FEATURES = 200000
# this will specify the number of words in the vocalobary our dictnary will have i.e. how many words will be stored in dict. in the form of textvectorization.
initialize textvectorization layer
vectorizer = TextVectorization(max_tokens = MAX_FEATURES, #max words in out vocab should be this 
                               output_sequence_length = 1800, #maximum length of the sentence in the token
                               output_mode = 'int') 
teach our vectorizer our vocab i.e. train the model with our vocab
vectorizer.adapt(x.values)
#adapt will learn all our words inside our vocablory
#to see the effect of vectorizer we can simply try this out
vectorizer('Hello world, life is great')[:5]
#this will give us the values for each word of the above sentence in the form of array, so 288 is Hello and so on.
do above effect for all comments rather than just above sentence
vectorizer.get_vocabulary()
vectorized_text = vectorizer(x.values)
#hence after above vectorization our data now looks like this
#so we have total 159571 samples and 1800 words are tokenized as we mentioned before in limit 
#also we can observe in the output that if perticular sentence does not meet the limit here 1800 words then it will just fill 0's in place of it.
vectorized_text
create our dataset 
#create tensorflow data pipeline--> this will make it easy to train deep learning model and is mainly used for the data which cannot fit into the memory 

#MCSHBAP - map, chache, shuffel, batch, prefetch

dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y)) #create a dataset, we pass our data we created above and  also the vectorized text
dataset = dataset.cache() #caches our data 
dataset = dataset.shuffle(160000) #pass the argument as how large should be the buffer size
dataset = dataset.batch(16) #each batch is represented as series of 16 samples 
dataset = dataset.prefetch(8) #help prevents bottlenecks
grab data from dataset 
dataset.as_numpy_iterator().next()
#this will represent our comment/text/sentences in vectorized format with all the labels 
#unpack above data as 
batch_x, batch_y = dataset.as_numpy_iterator().next()
#vectorized text examples -->we can see we have 16 samples and maxed as 1800 words each 
batch_x.shape
#labels --> 16 samples represented by 6 values in the vector
batch_y.shape
create our training validation and test partetation 
train = dataset.take(int(len(dataset)*.7)) #take 70% of the length of our dataset i.e. assign 70% of the data to training partetion.
# len(dataset) will print the length of data(it will be in batches since we converted before). 
# len(dataset)*.7 will give 70% of that data. 
#int(len(dataset)*.7) will convert it into integer
#dataset.take will take this pertationed data out and assign it to variable train
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
#.skip specifies that skip 70% of the data i.e. used in train and then take another 20% as our validation partation
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))
#here skip 70+20 = 90% of the data and remaining 1% used as testing data 
train.as_numpy_iterator().next()
#take the batch out of data as dataset name (train here).as_numpy_iterator which will create a generator and .next will get us the next batch 
Embedding the tokens.
It's like a personality test for a word. It tells us all about that word which will make it very useful for deep learnning

1st step is to create sequential model
from tensorflow.keras.models import Sequential
#first import the sequential APIs since it is fastest and easiest of all other methods
from tensorflow.keras.layers import Input, LSTM, Dropout, Bidirectional, Dense, Embedding
#import the layers which we will use to build on our deep neuron network
#first layer will be lstm layer and bi-directional will be a modifier on top of it and will allow us to pass the features/values from lstm output across the board as we pass through the sequences 
#dropout is method of regularization and dense layer is our fully connected layer 
# Define the input shape
INPUT_LENGTH =  None
input_layer = Input(shape=(INPUT_LENGTH ,))

model = Sequential() #this will instantiate the sequrntial APIs and then we can add in the number of layers we need to build it up

# Add the Input layer
model.add(input_layer)

#create the embedding layer, embedding is represented as number of words+1  +1 for unknow word. Our embedding will be 32 in length. i.e 32 features in the embedding for each word 
model.add(Embedding(MAX_FEATURES+1, 32))
#create LSTM layer which will have 32 different lstm units and specify that they have activation of tan-h 
model.add(Bidirectional(LSTM(32, activation='tanh')))

#3 layers for feature extraction
model.add(Dense(128, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))

#final layers that will map to different output we got inside our neural network
model.add(Dense(6, activation = 'sigmoid'))

y[0]
model.compile(loss='BinaryCrossentropy', optimizer='Adam')
model.summary()
create the training set 
history = model.fit(train, epochs=10, validation_data=val)
#for training we run this .fit and pass training data, epochs specifies how long we want to train it or how many passes we are going to do for our training data and also pass validation data 
history.history
from matplotlib import pyplot as plt
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()
Make predictions
input_text = vectorizer('You freaking suck!')
#use vectorizer and pass the text we want to vectorize 
#input text will be a sequence of integers which will be 1800 value long and if it does not meet with this length it will pad the results with 0
now when we pass this to .predict as model.predict(input_text) then it will not work since we have not passed the input as a batch or series of values. 

since the model will expect the output to be like 
none x no. of val in our sequence x length of embedding/ input shape 
#create a batch variable to pass input in batch
batch = test.as_numpy_iterator().next()
#get data out on data pipeline
batch_x, batch_y =  test.as_numpy_iterator().next()
model.predict(batch_y)
#here we can see all our comments are predicted.
# to see only the ones with value 1 we can do 
(model.predict(batch_y) > 0.5).astype(int)
#hence we pass it on custom dataset as 
model.predict(np.expand_dims(input_text,0))
res = model.predict(np.expand_dims(input_text,0))
# res will store our result and .predict will be used to make prediction
evaluating the model
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy
#instiantiate precision matrix, recall and categorical accuracy 
pre = Precision() 
re = Recall()
acc = CategoricalAccuracy()
#making these predictions 

#loop through every single batch inside out data pipeline
for batch in test.as_numpy_iterator():
    #unpack the batch/values
    x_true, y_true = batch

    #make predictin and pass our comments x_true which are already tokenize 
    yhat = model.predict(x_true)

    #flatten the prediction so that it become one big vector 
    y_true= y_true.flatten()
    yhat = yhat.flatten()

    #update the metrices based on current state of data
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)
print(f'Precision: {pre.result().numpy()}, Recall: {re.result().numpy()},Accuracy:{acc.result().numpy()}')
build the gradio app
%pip install gradio jinja2
import gradio as gr
model.save('Toxic_Comments.h5')
#save the model in a file names Toxic_Comments in h5 format
model = tf.keras.models.load_model('Toxic_Comments.h5')
# load our model as backup 
input_str = vectorizer('hey I frekin hate you! I am coming for you and I will hurt you')
res = model.predict(np.expand_dims( input_str, 0))
df.columns[2:]
res
#implement a function that we will hook into our gradio model

def score_comment(comment):
    #pass the comment  through vectorizer so that it convert it into series of numbers
    vectorized_comment = vectorizer([comment])
    #run comments through our model
    results = model.predict(vectorized_comment)

    #unpack all our results 
    #this loop will go through each of our columns toxic, obscence etc one by one and result will be printed
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx] > 0.5)
    
    return text
interface = gr.Interface(fn = score_comment, #pass the function we want to run when we hit enter 
                         inputs = gr.Textbox(lines=2, placeholder = 'comment to score'), #input will be textbox 
                         outputs = 'text') #output will be set of text 
interface.launch(share=True)